In [66]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing import sequence, text
from keras.layers import Input, Embedding

from nltk import word_tokenize
from nltk.corpus import stopwords
from textblob import TextBlob

import datetime as dt
import pandas as pd
import numpy as np
import warnings
import string

# stop_words = []
stop_words = list(set(stopwords.words('english')))
warnings.filterwarnings('ignore')
punctuation = string.punctuation
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ecommerce/Train.csv
/kaggle/input/ecommerce/Sample_Submission.xlsx
/kaggle/input/ecommerce/Test.csv


In [67]:
import pandas as pd
import numpy as np # linear algebra
import pandas as pd 
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import time
from sklearn.model_selection import *
from sklearn.metrics import *
from sklearn.linear_model import *
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import TreebankWordTokenizer
import matplotlib.pyplot as plt
%matplotlib inline
import re
import numpy as np

In [68]:
train = pd.read_csv(r'/kaggle/input/ecommerce/Train.csv')
test = pd.read_csv(r'/kaggle/input/ecommerce/Test.csv')

In [69]:
train.head()

,Product,Product_Brand,Item_Category,Subcategory_1,Subcategory_2,Item_Rating,Date,Selling_Price
0,P-2610,B-659,bags wallets belts,bags,hand bags,4.3,2/3/2017,291.0
1,P-2453,B-3078,clothing,women s clothing,western wear,3.1,7/1/2015,897.0
2,P-6802,B-1810,home decor festive needs,showpieces,ethnic,3.5,1/12/2019,792.0
3,P-4452,B-3078,beauty and personal care,eye care,h2o plus eye care,4.0,12/12/2014,837.0
4,P-8454,B-3078,clothing,men s clothing,t shirts,4.3,12/12/2013,470.0


In [70]:
train.drop(['Product'],inplace=True,axis=1)

In [71]:
test.drop(['Product'],inplace=True,axis=1)

In [72]:
train['Item_Rating'] = train['Item_Rating'].round()
test['Item_Rating'] = test['Item_Rating'].round()

In [73]:
from fastai import *
#from fastai.structured import add_datepart

In [74]:
def add_datepart(df, fldname, drop=True):
    fld = df[fldname]
    if not np.issubdtype(fld.dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    for n in ('Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start'):
        df[targ_pre+n] = getattr(fld.dt,n.lower())
    df[targ_pre+'Elapsed'] = fld.astype(np.int64) // 10**9
    if drop: df.drop(fldname, axis=1, inplace=True)
        
        

In [75]:
df = pd.concat([train, test], axis=0).reset_index(drop=True)

In [76]:
df = pd.get_dummies(df, columns = ['Item_Rating','Product_Brand'])

In [77]:
#df.drop(['Subcategory_3'],inplace = True, axis=1)

In [78]:
df['Subcategory_3'] = df['Subcategory_1'].str.cat(df['Subcategory_2'],sep=" ")

In [79]:
df.drop(['Subcategory_1', 'Subcategory_2'],inplace=True,axis=1)

In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3503 entries, 0 to 3502
Columns: 1322 entries, Item_Category to Subcategory_3
dtypes: float64(1), object(3), uint8(1318)
memory usage: 4.5+ MB


In [81]:
add_datepart(df,'Date')

In [82]:
df.drop(['Elapsed'],inplace=True , axis=1)

In [83]:
#df.drop(['Item_Category'],inplace=True,axis=1)

In [84]:
df.head()

,Item_Category,Selling_Price,Item_Rating_1.0,Item_Rating_2.0,Item_Rating_3.0,Item_Rating_4.0,Item_Rating_5.0,Product_Brand_B-1,Product_Brand_B-1000,Product_Brand_B-1004,...,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start
0,bags wallets belts,291.0,0,0,0,1,0,0,0,0,...,5,3,4,34,False,False,False,False,False,False
1,clothing,897.0,0,0,1,0,0,0,0,0,...,27,1,2,182,False,True,False,True,False,False
2,home decor festive needs,792.0,0,0,0,1,0,0,0,0,...,2,12,5,12,False,False,False,False,False,False
3,beauty and personal care,837.0,0,0,0,1,0,0,0,0,...,50,12,4,346,False,False,False,False,False,False
4,clothing,470.0,0,0,0,1,0,0,0,0,...,50,12,3,346,False,False,False,False,False,False


In [85]:
df["Item_Category_Len"] = df['Item_Category'].apply(len)
df["Subcategory_3_len"] = df['Subcategory_3'].apply(len)


In [86]:
df['text'] = df.apply(lambda row: ' '.join([str(row['Item_Category']), 
                                           # str(row['project_essay_2']), 
                                           # str(row['project_essay_3']), 
                                            str(row['Subcategory_3'])]), axis=1)

# extract features from text
df['char_count'] = df['text'].apply(len)
df['word_count'] = df['text'].apply(lambda x: len(x.split()))
df['word_density'] = df['char_count'] / (df['word_count']+1)
df['punctuation_count'] = df['text'].apply(lambda x: len("".join(_ for _ in x if _ in punctuation))) 
df['title_word_count'] = df['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
df['upper_case_word_count'] = df['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))
df['stopword_count'] = df['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.lower() in stop_words]))

In [87]:
pos_dic = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

# function to check and get the part of speech tag count of a words in a given sentence
def pos_check(x, flag):
    cnt = 0
    try:
        wiki = TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_dic[flag]:
                cnt += 1
    except:
        pass
    return cnt

df['noun_count'] = df['text'].apply(lambda x: pos_check(x, 'noun'))
df['verb_count'] = df['text'].apply(lambda x: pos_check(x, 'verb'))
df['adj_count'] = df['text'].apply(lambda x: pos_check(x, 'adj'))
df['adv_count'] = df['text'].apply(lambda x: pos_check(x, 'adv'))
df['pron_count'] = df['text'].apply(lambda x: pos_check(x, 'pron'))

In [88]:
df['noun_count_Item'] = df['Item_Category'].apply(lambda x: pos_check(x, 'noun'))
df['verb_count_Item'] = df['Item_Category'].apply(lambda x: pos_check(x, 'verb'))
df['adj_count_Item'] = df['Item_Category'].apply(lambda x: pos_check(x, 'adj'))
df['adv_count_Item'] = df['Item_Category'].apply(lambda x: pos_check(x, 'adv'))
df['pron_count_Item'] = df['Item_Category'].apply(lambda x: pos_check(x, 'pron'))

In [89]:
df['noun_count_sub'] = df['Subcategory_3'].apply(lambda x: pos_check(x, 'noun'))
df['verb_count_sub'] = df['Subcategory_3'].apply(lambda x: pos_check(x, 'verb'))
df['adj_count_sub'] = df['Subcategory_3'].apply(lambda x: pos_check(x, 'adj'))
df['adv_count_sub'] = df['Subcategory_3'].apply(lambda x: pos_check(x, 'adv'))
df['pron_count_sub'] = df['Subcategory_3'].apply(lambda x: pos_check(x, 'pron'))

In [90]:

cvec = TfidfVectorizer(max_features=10000, norm = 'l1', lowercase=True, smooth_idf=False, sublinear_tf=False, ngram_range=(1,4), tokenizer=TreebankWordTokenizer().tokenize)
df_info = pd.DataFrame(cvec.fit_transform(df['Item_Category']).todense())
df_info.columns = ['Item_Category_Top_' + str(c) for c in df_info.columns]
df = pd.concat([df, df_info], axis=1)

In [91]:
cvec = TfidfVectorizer(max_features=10000, norm = 'l1', lowercase=True, smooth_idf=False, sublinear_tf=False, ngram_range=(1,4), tokenizer=TreebankWordTokenizer().tokenize)
df_info = pd.DataFrame(cvec.fit_transform(df['Subcategory_3']).todense())
df_info.columns = ['Subcategory_3_Top_' + str(c) for c in df_info.columns]
df = pd.concat([df, df_info], axis=1)

In [92]:
df.drop(['text'],inplace=True, axis=1)

In [93]:
train, test  = df[:train.shape[0]].reset_index(drop=True), df[train.shape[0]:].reset_index(drop=True)

In [94]:
train.head()

,Item_Category,Selling_Price,Item_Rating_1.0,Item_Rating_2.0,Item_Rating_3.0,Item_Rating_4.0,Item_Rating_5.0,Product_Brand_B-1,Product_Brand_B-1000,Product_Brand_B-1004,...,Subcategory_3_Top_3453,Subcategory_3_Top_3454,Subcategory_3_Top_3455,Subcategory_3_Top_3456,Subcategory_3_Top_3457,Subcategory_3_Top_3458,Subcategory_3_Top_3459,Subcategory_3_Top_3460,Subcategory_3_Top_3461,Subcategory_3_Top_3462
0,bags wallets belts,291.0,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,clothing,897.0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,home decor festive needs,792.0,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,beauty and personal care,837.0,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,clothing,470.0,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [95]:
train.drop(['Item_Category'],inplace=True, axis=1)

In [96]:
train.drop(['Subcategory_3'],inplace=True, axis=1)

In [99]:
#train.drop(['text'],inplace=True, axis=1)#

In [100]:
TARGET_COL =  'Selling_Price'
features = [c for c in train.columns if c not in [TARGET_COL]]
target = train[TARGET_COL]

In [101]:
#target = np.log1p(target)

In [102]:

def rmse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred) ** 0.5

In [103]:

training_start_time = time.time()

max_iter = 20
folds = StratifiedKFold(n_splits = max_iter)
oofs = np.zeros(len(train))
test_preds = np.zeros(len(test))


for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, pd.qcut(target, 10, labels=False, duplicates='drop'))):
    
    print(f'\n---- Fold {fold_} -----\n')
    
    fold_start_time = time.time()
    
    X_trn, y_trn = train.iloc[trn_idx][features], target.iloc[trn_idx]
    X_val, y_val = train.iloc[val_idx][features], target.iloc[val_idx]
    X_test = test[features]
    print(X_trn.shape[1], X_val.shape[1])
    
    
    clf = LGBMRegressor(n_estimators=1000, num_leaves=127, max_depth=-1,min_child_samples=4, learning_rate=0.02, colsample_bytree=0.4, reg_alpha=0.5, reg_lambda=2)
    _ = clf.fit(X_trn, np.log(y_trn), eval_set = [(X_val, np.log(y_val))], verbose=100, early_stopping_rounds=200, eval_metric='rmse')

    oofs[val_idx] = np.exp(clf.predict(X_val))
    current_test_pred = np.exp(clf.predict(X_test))
    test_preds += np.exp(clf.predict(X_test))/max_iter
    
    
    print(f'\n Fold {rmse(np.log(y_val), np.log(oofs[val_idx]))}')
    
    fold_end_time = time.time()
    total_fold_time = int(fold_end_time - fold_start_time)
    
    print(f"\n->-> Fold ran for {(total_fold_time)//60} minutes {(total_fold_time)%60} seconds")
    

print(f'\nOOF val score: {rmse(np.log(target), np.log(oofs))}')
training_end_time = time.time()
total_training_time = int(training_end_time - training_start_time)

print(f'\n->-> Total training time: {(total_training_time)//60} minutes {(total_training_time)%60} seconds')


---- Fold 0 -----

5955 5955
Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 0.665907	valid_0's l2: 0.443432
[200]	valid_0's rmse: 0.652267	valid_0's l2: 0.425452
[300]	valid_0's rmse: 0.655045	valid_0's l2: 0.429084
Early stopping, best iteration is:
[182]	valid_0's rmse: 0.650573	valid_0's l2: 0.423245

 Fold 0.650572598398268

->-> Fold ran for 0 minutes 6 seconds

---- Fold 1 -----

5955 5955
Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 0.697673	valid_0's l2: 0.486748
[200]	valid_0's rmse: 0.696907	valid_0's l2: 0.48568
[300]	valid_0's rmse: 0.705307	valid_0's l2: 0.497457
Early stopping, best iteration is:
[156]	valid_0's rmse: 0.693035	valid_0's l2: 0.480297

 Fold 0.6930346852169773

->-> Fold ran for 0 minutes 6 seconds

---- Fold 2 -----

5955 5955
Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 0.834314	valid_0's l2: 0.69608
[200]	valid_0's rmse: 0.81371	valid_0's l


 Fold 0.7462804363094185

->-> Fold ran for 0 minutes 8 seconds

---- Fold 18 -----

5955 5955
Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 0.751693	valid_0's l2: 0.565043
[200]	valid_0's rmse: 0.741048	valid_0's l2: 0.549152
[300]	valid_0's rmse: 0.737615	valid_0's l2: 0.544076
[400]	valid_0's rmse: 0.735241	valid_0's l2: 0.540579
[500]	valid_0's rmse: 0.73523	valid_0's l2: 0.540564
Early stopping, best iteration is:
[359]	valid_0's rmse: 0.734319	valid_0's l2: 0.539225

 Fold 0.734319379296395

->-> Fold ran for 0 minutes 7 seconds

---- Fold 19 -----

5955 5955
Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 0.595437	valid_0's l2: 0.354546
[200]	valid_0's rmse: 0.563969	valid_0's l2: 0.318061
[300]	valid_0's rmse: 0.568355	valid_0's l2: 0.323028
[400]	valid_0's rmse: 0.574146	valid_0's l2: 0.329644
Early stopping, best iteration is:
[216]	valid_0's rmse: 0.563012	valid_0's l2: 0.316982

 Fold 0.5630119642764

In [104]:
print(f'\nOOF val score: {rmse(np.log(target), np.log(oofs))}')


OOF val score: 0.6584012926203036


In [105]:
vp0, tp0 = oofs, test_preds

In [106]:
training_start_time = time.time()

max_iter = 10
folds = StratifiedKFold(n_splits = max_iter)
oofs = np.zeros(len(train))
test_preds = np.zeros(len(test))


for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, pd.qcut(target, 10, labels=False, duplicates='drop'))):
    
    print(f'\n---- Fold {fold_} -----\n')
    
    fold_start_time = time.time()
    
    X_trn, y_trn = train.iloc[trn_idx][features], target.iloc[trn_idx]
    X_val, y_val = train.iloc[val_idx][features], target.iloc[val_idx]
    X_test = test[features]
    print(X_trn.shape[1], X_val.shape[1])
    

    clf = XGBRegressor(n_estimators=3000, max_depth=12, learning_rate=0.05, colsample_bytree=0.9,subsample=1)
    _ = clf.fit(X_trn, np.log(y_trn), eval_set = [(X_val, np.log(y_val))], verbose=100, early_stopping_rounds=200, eval_metric='rmse')
    

    oofs[val_idx] = np.exp(clf.predict(X_val))
    current_test_pred = np.exp(clf.predict(X_test))
    test_preds += np.exp(clf.predict(X_test))/max_iter
    
    
    print(f'\n Fold {rmse(np.log(y_val), np.log(oofs[val_idx]))}')
    
    fold_end_time = time.time()
    total_fold_time = int(fold_end_time - fold_start_time)
    
    print(f"\n->-> Fold ran for {(total_fold_time)//60} minutes {(total_fold_time)%60} seconds")
    

print(f'\nOOF val score: {rmse(np.log(target), np.log(oofs))}')
training_end_time = time.time()
total_training_time = int(training_end_time - training_start_time)

print(f'\n->-> Total training time: {(total_training_time)//60} minutes {(total_training_time)%60} seconds')



---- Fold 0 -----

5955 5955
[0]	validation_0-rmse:6.01196
Will train until validation_0-rmse hasn't improved in 200 rounds.
[100]	validation_0-rmse:0.64306
[200]	validation_0-rmse:0.63759
[300]	validation_0-rmse:0.63309
[400]	validation_0-rmse:0.63062
[500]	validation_0-rmse:0.63040
[600]	validation_0-rmse:0.63011
[700]	validation_0-rmse:0.62899
[800]	validation_0-rmse:0.62917
[900]	validation_0-rmse:0.62960
Stopping. Best iteration:
[703]	validation_0-rmse:0.62876


 Fold 0.6287636039275231

->-> Fold ran for 5 minutes 38 seconds

---- Fold 1 -----

5955 5955
[0]	validation_0-rmse:5.95588
Will train until validation_0-rmse hasn't improved in 200 rounds.
[100]	validation_0-rmse:0.75758
[200]	validation_0-rmse:0.74993
[300]	validation_0-rmse:0.74711
[400]	validation_0-rmse:0.74774
[500]	validation_0-rmse:0.74497
[600]	validation_0-rmse:0.74332
[700]	validation_0-rmse:0.74283
[800]	validation_0-rmse:0.74247
[900]	validation_0-rmse:0.74169
[1000]	validation_0-rmse:0.74185
[1100]	validat

In [107]:
vp1, tp1 = oofs, test_preds

In [131]:
training_start_time = time.time()

max_iter = 10
folds = StratifiedKFold(n_splits = max_iter)
oofs = np.zeros(len(train))
test_preds = np.zeros(len(test))


for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, pd.qcut(target, 10, labels=False, duplicates='drop'))):
    
    print(f'\n---- Fold {fold_} -----\n')
    
    fold_start_time = time.time()
    
    X_trn, y_trn = train.iloc[trn_idx][features], target.iloc[trn_idx]
    X_val, y_val = train.iloc[val_idx][features], target.iloc[val_idx]
    X_test = test[features]
    print(X_trn.shape[1], X_val.shape[1])
    
    
    
    clf = CatBoostRegressor(n_estimators=2000, learning_rate=0.05, max_depth=10, rsm=0.5)
    _ = clf.fit(X_trn, np.log(y_trn), eval_set = [(X_val, np.log(y_val))], verbose=100, early_stopping_rounds=200)

    oofs[val_idx] = np.exp(clf.predict(X_val))
    current_test_pred = np.exp(clf.predict(X_test))
    test_preds += np.exp(clf.predict(X_test))/max_iter
    
    
    
    print(f'\n Fold {rmse(np.log(y_val), np.log(oofs[val_idx]))}')
    
    fold_end_time = time.time()
    total_fold_time = int(fold_end_time - fold_start_time)
    
    print(f"\n->-> Fold ran for {(total_fold_time)//60} minutes {(total_fold_time)%60} seconds")
    

print(f'\nOOF val score: {rmse(np.log(target), np.log(oofs))}')
training_end_time = time.time()
total_training_time = int(training_end_time - training_start_time)

print(f'\n->-> Total training time: {(total_training_time)//60} minutes {(total_training_time)%60} seconds')



---- Fold 0 -----

5955 5955
0:	learn: 1.1431241	test: 1.2117389	best: 1.2117389 (0)	total: 167ms	remaining: 5m 34s
100:	learn: 0.6718573	test: 0.6941477	best: 0.6941477 (100)	total: 7.46s	remaining: 2m 20s
200:	learn: 0.6070368	test: 0.6777321	best: 0.6777321 (200)	total: 14.6s	remaining: 2m 10s
300:	learn: 0.5436266	test: 0.6574582	best: 0.6574582 (300)	total: 21.9s	remaining: 2m 3s
400:	learn: 0.5037531	test: 0.6477544	best: 0.6477544 (400)	total: 29.3s	remaining: 1m 56s
500:	learn: 0.4751833	test: 0.6385161	best: 0.6385161 (500)	total: 36.7s	remaining: 1m 49s
600:	learn: 0.4504002	test: 0.6340693	best: 0.6340693 (600)	total: 44s	remaining: 1m 42s
700:	learn: 0.4304151	test: 0.6306256	best: 0.6306256 (700)	total: 51.3s	remaining: 1m 34s
800:	learn: 0.4119985	test: 0.6277611	best: 0.6277550 (795)	total: 59.1s	remaining: 1m 28s
900:	learn: 0.3946195	test: 0.6256645	best: 0.6254259 (898)	total: 1m 6s	remaining: 1m 21s
1000:	learn: 0.3785758	test: 0.6237321	best: 0.6235846 (986)	total:

Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.6918954272
bestIteration = 1534

Shrink model to first 1535 iterations.

 Fold 0.6918954372459585

->-> Fold ran for 2 minutes 11 seconds

---- Fold 5 -----

5955 5955
0:	learn: 1.1498982	test: 1.1455924	best: 1.1455924 (0)	total: 87.9ms	remaining: 2m 55s
100:	learn: 0.6820876	test: 0.6649821	best: 0.6649760 (99)	total: 7.59s	remaining: 2m 22s
200:	learn: 0.6140598	test: 0.6338910	best: 0.6338910 (200)	total: 15.2s	remaining: 2m 15s
300:	learn: 0.5479430	test: 0.6082189	best: 0.6082188 (299)	total: 23.1s	remaining: 2m 10s
400:	learn: 0.5073639	test: 0.5989463	best: 0.5989463 (400)	total: 30.6s	remaining: 2m 2s
500:	learn: 0.4795625	test: 0.5953051	best: 0.5952368 (498)	total: 38.2s	remaining: 1m 54s
600:	learn: 0.4560825	test: 0.5888149	best: 0.5888149 (600)	total: 45.7s	remaining: 1m 46s
700:	learn: 0.4349785	test: 0.5880721	best: 0.5880676 (699)	total: 53.2s	remaining: 1m 38s
800:	learn: 0.4143724	test: 0.5849458	be

800:	learn: 0.4077203	test: 0.6281755	best: 0.6280826 (796)	total: 1m 1s	remaining: 1m 32s
900:	learn: 0.3910843	test: 0.6275537	best: 0.6268720 (868)	total: 1m 9s	remaining: 1m 24s
1000:	learn: 0.3763801	test: 0.6262105	best: 0.6259799 (990)	total: 1m 16s	remaining: 1m 16s
1100:	learn: 0.3646014	test: 0.6248624	best: 0.6248624 (1100)	total: 1m 24s	remaining: 1m 9s
1200:	learn: 0.3510449	test: 0.6237150	best: 0.6236621 (1198)	total: 1m 32s	remaining: 1m 1s
1300:	learn: 0.3380112	test: 0.6227617	best: 0.6225033 (1294)	total: 1m 40s	remaining: 54s
1400:	learn: 0.3282210	test: 0.6217040	best: 0.6214462 (1358)	total: 1m 48s	remaining: 46.2s
1500:	learn: 0.3173659	test: 0.6201399	best: 0.6200327 (1488)	total: 1m 55s	remaining: 38.5s
1600:	learn: 0.3087850	test: 0.6207675	best: 0.6198276 (1535)	total: 2m 3s	remaining: 30.7s
1700:	learn: 0.3019848	test: 0.6211191	best: 0.6198276 (1535)	total: 2m 10s	remaining: 23s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.6198276102

In [ ]:
vp2, tp2 = oofs, test_preds

In [163]:
params={'learning_rate': 0.01,
        'objective':'regression',
        'metric':'rmse',
        'num_leaves': 31,
        'verbose': 1,
        'bagging_fraction': 0.9,
        'feature_fraction': 0.9,
        "random_state":42,
        'max_depth': 5,
        "bagging_seed" : 42,
        "verbosity" : -1,
        "bagging_frequency" : 5,
        'lambda_l2': 0.5,
        'lambda_l1': 0.5,
        'min_child_samples': 36
       }

cat_param = {
    'learning_rate' :0.03,
    'depth' :10,
    'eval_metric' :'RMSE',
    'od_type' :'Iter',
    'metric_period ' : 50,
    'od_wait' : 20,
    'seed' : 42
    
}


In [164]:
training_start_time = time.time()

max_iter = 10
folds = StratifiedKFold(n_splits = max_iter)
oofs = np.zeros(len(train))
test_preds = np.zeros(len(test))


for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, pd.qcut(target, 10, labels=False, duplicates='drop'))):
    
    print(f'\n---- Fold {fold_} -----\n')
    
    fold_start_time = time.time()
    
    X_trn, y_trn = train.iloc[trn_idx][features], target.iloc[trn_idx]
    X_val, y_val = train.iloc[val_idx][features], target.iloc[val_idx]
    X_test = test[features]
    print(X_trn.shape[1], X_val.shape[1])
    
    
    
    clf = CatBoostRegressor(iterations=1000,learning_rate=0.03,
                            depth=10,
                            eval_metric='RMSE',
                            random_seed = 42,
                            bagging_temperature = 0.2,
                            od_type='Iter',
                            metric_period = 50,
                            od_wait=20)
    _ = clf.fit(X_trn, np.log(y_trn), eval_set = [(X_val, np.log(y_val))], verbose=100, early_stopping_rounds=200)

    oofs[val_idx] = np.exp(clf.predict(X_val))
    current_test_pred = np.exp(clf.predict(X_test))
    test_preds += np.exp(clf.predict(X_test))/max_iter
    
    
    
    print(f'\n Fold {rmse(np.log(y_val), np.log(oofs[val_idx]))}')
    
    fold_end_time = time.time()
    total_fold_time = int(fold_end_time - fold_start_time)
    
    print(f"\n->-> Fold ran for {(total_fold_time)//60} minutes {(total_fold_time)%60} seconds")
    

print(f'\nOOF val score: {rmse(np.log(target), np.log(oofs))}')
training_end_time = time.time()
total_training_time = int(training_end_time - training_start_time)

print(f'\n->-> Total training time: {(total_training_time)//60} minutes {(total_training_time)%60} seconds')


---- Fold 0 -----

5955 5955


0:	learn: 1.1532072	test: 1.2245647	best: 1.2245647 (0)	total: 178ms	remaining: 2m 57s
100:	learn: 0.6981829	test: 0.7048990	best: 0.7048990 (100)	total: 7.53s	remaining: 1m 7s
200:	learn: 0.6499107	test: 0.6856786	best: 0.6856786 (200)	total: 15.2s	remaining: 1m
300:	learn: 0.6113958	test: 0.6755989	best: 0.6755621 (299)	total: 22.7s	remaining: 52.7s
400:	learn: 0.5666016	test: 0.6604304	best: 0.6602864 (399)	total: 30.1s	remaining: 45s
500:	learn: 0.5344441	test: 0.6517777	best: 0.6517777 (499)	total: 37.5s	remaining: 37.4s
600:	learn: 0.5107962	test: 0.6464289	best: 0.6464289 (600)	total: 45s	remaining: 29.9s
700:	learn: 0.4891697	test: 0.6419013	best: 0.6419013 (700)	total: 52.5s	remaining: 22.4s
800:	learn: 0.4727435	test: 0.6389870	best: 0.6389865 (799)	total: 60s	remaining: 14.9s
900:	learn: 0.4562354	test: 0.6366352	best: 0.6366152 (896)	total: 1m 7s	remaining: 7.4s
999:	learn: 0.4395453	test: 0.6334917	best: 0.6334852 (995)	total: 1m 15s	remaining: 0us

bestTest = 0.6334852222

0:	learn: 1.1603786	test: 1.1701198	best: 1.1701198 (0)	total: 136ms	remaining: 2m 16s
100:	learn: 0.6812190	test: 0.8043293	best: 0.8043293 (100)	total: 7.39s	remaining: 1m 5s
200:	learn: 0.6357312	test: 0.7894028	best: 0.7894008 (199)	total: 14.6s	remaining: 57.9s
300:	learn: 0.6006107	test: 0.7802306	best: 0.7799465 (298)	total: 21.7s	remaining: 50.5s
400:	learn: 0.5543528	test: 0.7682455	best: 0.7682455 (400)	total: 29s	remaining: 43.3s
500:	learn: 0.5259006	test: 0.7628797	best: 0.7628797 (500)	total: 36.3s	remaining: 36.1s
600:	learn: 0.4995845	test: 0.7592809	best: 0.7592809 (600)	total: 43.5s	remaining: 28.8s
700:	learn: 0.4783643	test: 0.7576904	best: 0.7574586 (688)	total: 50.7s	remaining: 21.6s
800:	learn: 0.4616630	test: 0.7552733	best: 0.7552073 (798)	total: 58.3s	remaining: 14.5s
900:	learn: 0.4467011	test: 0.7527582	best: 0.7527556 (899)	total: 1m 5s	remaining: 7.22s
999:	learn: 0.4319787	test: 0.7515813	best: 0.7515813 (999)	total: 1m 12s	remaining: 0us

bestTest = 0.75

0:	learn: 1.1603771	test: 1.1724026	best: 1.1724026 (0)	total: 91.6ms	remaining: 1m 31s
100:	learn: 0.7073205	test: 0.6956682	best: 0.6956682 (100)	total: 7.39s	remaining: 1m 5s
200:	learn: 0.6560342	test: 0.6696439	best: 0.6696439 (200)	total: 14.8s	remaining: 58.8s
300:	learn: 0.6176253	test: 0.6569589	best: 0.6569589 (300)	total: 22.2s	remaining: 51.6s
400:	learn: 0.5724228	test: 0.6366673	best: 0.6366673 (400)	total: 30.1s	remaining: 44.9s
500:	learn: 0.5375045	test: 0.6249845	best: 0.6249845 (499)	total: 37.8s	remaining: 37.7s
600:	learn: 0.5135288	test: 0.6194236	best: 0.6194236 (600)	total: 45.6s	remaining: 30.3s
700:	learn: 0.4939005	test: 0.6154974	best: 0.6152752 (686)	total: 53.3s	remaining: 22.7s
800:	learn: 0.4769831	test: 0.6121541	best: 0.6121491 (797)	total: 1m	remaining: 15.1s
900:	learn: 0.4585539	test: 0.6101323	best: 0.6099817 (874)	total: 1m 8s	remaining: 7.52s
999:	learn: 0.4456612	test: 0.6086198	best: 0.6086198 (999)	total: 1m 15s	remaining: 0us

bestTest = 0.60

0:	learn: 1.1692505	test: 1.0795587	best: 1.0795587 (0)	total: 86.9ms	remaining: 1m 26s
100:	learn: 0.7071453	test: 0.6556991	best: 0.6556991 (100)	total: 7.1s	remaining: 1m 3s
200:	learn: 0.6613467	test: 0.6347263	best: 0.6347196 (199)	total: 14.1s	remaining: 56s
300:	learn: 0.6231144	test: 0.6230667	best: 0.6230548 (294)	total: 21.4s	remaining: 49.8s
400:	learn: 0.5763845	test: 0.6066716	best: 0.6066716 (400)	total: 28.8s	remaining: 43s
500:	learn: 0.5385792	test: 0.5942566	best: 0.5941524 (499)	total: 36s	remaining: 35.9s
600:	learn: 0.5132530	test: 0.5893477	best: 0.5893053 (598)	total: 43.2s	remaining: 28.7s
700:	learn: 0.4918145	test: 0.5845717	best: 0.5845717 (700)	total: 50.6s	remaining: 21.6s
800:	learn: 0.4722867	test: 0.5825561	best: 0.5825561 (800)	total: 57.8s	remaining: 14.4s
900:	learn: 0.4557347	test: 0.5798349	best: 0.5797660 (894)	total: 1m 4s	remaining: 7.14s
999:	learn: 0.4421496	test: 0.5761956	best: 0.5761956 (999)	total: 1m 12s	remaining: 0us

bestTest = 0.576195

...

0:	learn: 1.1624534	test: 1.1583127	best: 1.1583127 (0)	total: 79ms	remaining: 1m 18s
100:	learn: 0.7081151	test: 0.6830739	best: 0.6830739 (100)	total: 7.72s	remaining: 1m 8s
200:	learn: 0.6587743	test: 0.6511875	best: 0.6511855 (199)	total: 15.2s	remaining: 1m
300:	learn: 0.6185448	test: 0.6360657	best: 0.6360656 (299)	total: 22.7s	remaining: 52.8s
400:	learn: 0.5748330	test: 0.6171524	best: 0.6171342 (399)	total: 30.5s	remaining: 45.5s
500:	learn: 0.5429023	test: 0.6095721	best: 0.6095291 (497)	total: 38.2s	remaining: 38s
600:	learn: 0.5184824	test: 0.6045352	best: 0.6044892 (599)	total: 46.9s	remaining: 31.1s
700:	learn: 0.4942749	test: 0.6004455	best: 0.6004455 (700)	total: 54.7s	remaining: 23.3s
800:	learn: 0.4773211	test: 0.5971956	best: 0.5970617 (788)	total: 1m 2s	remaining: 15.5s
900:	learn: 0.4629633	test: 0.5957711	best: 0.5955526 (893)	total: 1m 10s	remaining: 7.71s
999:	learn: 0.4435222	test: 0.5932919	best: 0.5932502 (998)	total: 1m 17s	remaining: 0us

bestTest = 0.59325

0:	learn: 1.1662493	test: 1.1107594	best: 1.1107594 (0)	total: 88.7ms	remaining: 1m 28s
100:	learn: 0.7038441	test: 0.7073685	best: 0.7073685 (100)	total: 7.54s	remaining: 1m 7s
200:	learn: 0.6554613	test: 0.6805015	best: 0.6805015 (200)	total: 15s	remaining: 59.7s
300:	learn: 0.6180388	test: 0.6687314	best: 0.6687265 (299)	total: 22.5s	remaining: 52.4s
400:	learn: 0.5709573	test: 0.6524010	best: 0.6524010 (400)	total: 30.4s	remaining: 45.4s
500:	learn: 0.5363810	test: 0.6416067	best: 0.6414992 (496)	total: 38.1s	remaining: 37.9s
600:	learn: 0.5117616	test: 0.6364772	best: 0.6364772 (600)	total: 46s	remaining: 30.5s
700:	learn: 0.4873498	test: 0.6321122	best: 0.6320377 (693)	total: 53.6s	remaining: 22.9s
800:	learn: 0.4658722	test: 0.6269160	best: 0.6269160 (800)	total: 1m 1s	remaining: 15.2s
900:	learn: 0.4488774	test: 0.6234153	best: 0.6232752 (895)	total: 1m 8s	remaining: 7.55s
999:	learn: 0.4319148	test: 0.6208258	best: 0.6207376 (997)	total: 1m 16s	remaining: 0us

bestTest = 0.620

0:	learn: 1.1582036	test: 1.1981059	best: 1.1981059 (0)	total: 78.8ms	remaining: 1m 18s
100:	learn: 0.6982653	test: 0.7491703	best: 0.7491703 (100)	total: 7.61s	remaining: 1m 7s
200:	learn: 0.6529128	test: 0.7251137	best: 0.7250008 (195)	total: 15s	remaining: 59.5s
300:	learn: 0.6138638	test: 0.7053022	best: 0.7052229 (299)	total: 22.3s	remaining: 51.8s
400:	learn: 0.5669717	test: 0.6761477	best: 0.6761477 (400)	total: 29.7s	remaining: 44.3s
500:	learn: 0.5340554	test: 0.6600141	best: 0.6600141 (500)	total: 37.1s	remaining: 36.9s
600:	learn: 0.5092773	test: 0.6513892	best: 0.6513892 (600)	total: 44.4s	remaining: 29.5s
700:	learn: 0.4885678	test: 0.6418048	best: 0.6418048 (700)	total: 51.7s	remaining: 22s
800:	learn: 0.4701006	test: 0.6351977	best: 0.6351388 (796)	total: 59s	remaining: 14.7s
900:	learn: 0.4542211	test: 0.6313106	best: 0.6313106 (900)	total: 1m 6s	remaining: 7.32s
999:	learn: 0.4409248	test: 0.6277841	best: 0.6277291 (998)	total: 1m 14s	remaining: 0us

bestTest = 0.62772

0:	learn: 1.1610555	test: 1.1538346	best: 1.1538346 (0)	total: 82.8ms	remaining: 1m 22s
100:	learn: 0.6924359	test: 0.7666085	best: 0.7666085 (100)	total: 7.35s	remaining: 1m 5s
200:	learn: 0.6450632	test: 0.7461265	best: 0.7461265 (200)	total: 14.6s	remaining: 58s
300:	learn: 0.6052784	test: 0.7322666	best: 0.7321301 (297)	total: 22.1s	remaining: 51.3s
400:	learn: 0.5577096	test: 0.7189315	best: 0.7189315 (400)	total: 29.4s	remaining: 44s
500:	learn: 0.5224764	test: 0.7098483	best: 0.7098483 (500)	total: 36.8s	remaining: 36.6s
600:	learn: 0.4982747	test: 0.7044236	best: 0.7044236 (600)	total: 44.3s	remaining: 29.4s
700:	learn: 0.4779387	test: 0.7004244	best: 0.7004244 (700)	total: 52s	remaining: 22.2s
800:	learn: 0.4587045	test: 0.6958106	best: 0.6955533 (792)	total: 59.7s	remaining: 14.8s
900:	learn: 0.4431286	test: 0.6922153	best: 0.6921845 (878)	total: 1m 7s	remaining: 7.37s
999:	learn: 0.4289944	test: 0.6905555	best: 0.6903433 (969)	total: 1m 14s	remaining: 0us

bestTest = 0.69034

0:	learn: 1.1550354	test: 1.2146964	best: 1.2146964 (0)	total: 82.8ms	remaining: 1m 22s
100:	learn: 0.7002268	test: 0.7284320	best: 0.7284320 (100)	total: 7.68s	remaining: 1m 8s
200:	learn: 0.6506626	test: 0.7036550	best: 0.7036550 (200)	total: 15.4s	remaining: 1m 1s
300:	learn: 0.6139504	test: 0.6885615	best: 0.6885615 (300)	total: 22.9s	remaining: 53.3s
400:	learn: 0.5648602	test: 0.6645954	best: 0.6645954 (400)	total: 30.5s	remaining: 45.6s
500:	learn: 0.5293623	test: 0.6525662	best: 0.6525662 (500)	total: 38.6s	remaining: 38.4s
600:	learn: 0.5045527	test: 0.6462824	best: 0.6462824 (600)	total: 46.5s	remaining: 30.9s
700:	learn: 0.4835746	test: 0.6402145	best: 0.6402145 (700)	total: 54.2s	remaining: 23.1s
800:	learn: 0.4659509	test: 0.6362535	best: 0.6362526 (799)	total: 1m 1s	remaining: 15.3s
900:	learn: 0.4512502	test: 0.6342233	best: 0.6340136 (893)	total: 1m 9s	remaining: 7.63s
999:	learn: 0.4372222	test: 0.6312821	best: 0.6312821 (999)	total: 1m 17s	remaining: 0us

bestTest = 0

In [165]:
vp3, tp3 = oofs, test_preds

In [166]:

training_start_time = time.time()

max_iter = 20
folds = StratifiedKFold(n_splits = max_iter)
oofs = np.zeros(len(train))
test_preds = np.zeros(len(test))


for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, pd.qcut(target, 10, labels=False, duplicates='drop'))):
    
    print(f'\n---- Fold {fold_} -----\n')
    
    fold_start_time = time.time()
    
    X_trn, y_trn = train.iloc[trn_idx][features], target.iloc[trn_idx]
    X_val, y_val = train.iloc[val_idx][features], target.iloc[val_idx]
    X_test = test[features]
    print(X_trn.shape[1], X_val.shape[1])
    
    
    #clf = LGBMRegressor(n_estimators=1000, num_leaves=127, max_depth=-1,min_child_samples=4, learning_rate=0.02, colsample_bytree=0.4, reg_alpha=0.5, reg_lambda=2)
    clf = LGBMRegressor(**params,n_estimators=1100)
    _ = clf.fit(X_trn, np.log(y_trn), eval_set = [(X_val, np.log(y_val))], verbose=100, early_stopping_rounds=200, eval_metric='rmse')

    oofs[val_idx] = np.exp(clf.predict(X_val))
    current_test_pred = np.exp(clf.predict(X_test))
    test_preds += np.exp(clf.predict(X_test))/max_iter
    
    
    print(f'\n Fold {rmse(np.log(y_val), np.log(oofs[val_idx]))}')
    
    fold_end_time = time.time()
    total_fold_time = int(fold_end_time - fold_start_time)
    
    print(f"\n->-> Fold ran for {(total_fold_time)//60} minutes {(total_fold_time)%60} seconds")
    

print(f'\nOOF val score: {rmse(np.log(target), np.log(oofs))}')
training_end_time = time.time()
total_training_time = int(training_end_time - training_start_time)

print(f'\n->-> Total training time: {(total_training_time)//60} minutes {(total_training_time)%60} seconds')


---- Fold 0 -----

5955 5955
Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 0.817926
[200]	valid_0's rmse: 0.69717
[300]	valid_0's rmse: 0.660329
[400]	valid_0's rmse: 0.650743
[500]	valid_0's rmse: 0.642567
[600]	valid_0's rmse: 0.63678
[700]	valid_0's rmse: 0.638605
[800]	valid_0's rmse: 0.63863
Early stopping, best iteration is:
[633]	valid_0's rmse: 0.635271

 Fold 0.6352712366996073

->-> Fold ran for 0 minutes 5 seconds

---- Fold 1 -----

5955 5955
Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 0.935327
[200]	valid_0's rmse: 0.861931
[300]	valid_0's rmse: 0.831636
[400]	valid_0's rmse: 0.806751
[500]	valid_0's rmse: 0.789842
[600]	valid_0's rmse: 0.776548
[700]	valid_0's rmse: 0.759778
[800]	valid_0's rmse: 0.746812
[900]	valid_0's rmse: 0.733454
[1000]	valid_0's rmse: 0.724997
[1100]	valid_0's rmse: 0.716986
Did not meet early stopping. Best iteration is:
[1100]	valid_0's rmse: 0.716986

 Fold 0.71698631

Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 0.93083
[200]	valid_0's rmse: 0.856927
[300]	valid_0's rmse: 0.820039
[400]	valid_0's rmse: 0.794883
[500]	valid_0's rmse: 0.77722
[600]	valid_0's rmse: 0.764827
[700]	valid_0's rmse: 0.752347
[800]	valid_0's rmse: 0.743308
[900]	valid_0's rmse: 0.738277
[1000]	valid_0's rmse: 0.735426
[1100]	valid_0's rmse: 0.729049
Did not meet early stopping. Best iteration is:
[1098]	valid_0's rmse: 0.728983

 Fold 0.728982818383847

->-> Fold ran for 0 minutes 5 seconds

---- Fold 16 -----

5955 5955
Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 0.804495
[200]	valid_0's rmse: 0.748252
[300]	valid_0's rmse: 0.730328
[400]	valid_0's rmse: 0.721479
[500]	valid_0's rmse: 0.720015
[600]	valid_0's rmse: 0.720873
Early stopping, best iteration is:
[457]	valid_0's rmse: 0.719185

 Fold 0.7191851050497591

->-> Fold ran for 0 minutes 5 seconds

---- Fold 17 -----

5955 5955
Training unt

In [167]:
vp4, tp4 = oofs, test_preds

In [193]:
##vp0*0.1 + vp1*0.2 + vp2*0.7  0.62

test_preds =  tp0*0.1  + tp1*0.4 + tp2*0.6

In [194]:
print(f'\nOOF val score: {rmse(np.log(target), np.log(oofs))}')


OOF val score: 0.7004693645723346


In [195]:

print(f'\nOOF val score: {rmse(np.log(target), np.log( vp1*0.2 + vp2*0.7 + vp3*0.1 ))}')


OOF val score: 0.6291850635550152


In [196]:
sub = pd.DataFrame({'Selling_Price': test_preds})
sub['Selling_Price'] = np.clip(sub['Selling_Price'], target.min(), target.max())
sub['Selling_Price'].describe()

count     1051.000000
mean      2014.203959
std       5865.736234
min        176.013813
25%        491.016208
50%        686.625497
75%        950.073945
max      52970.848329
Name: Selling_Price, dtype: float64

In [197]:

sub.head(10)

,Selling_Price
0,2730.792557
1,504.778122
2,832.701076
3,385.966062
4,791.196730
5,785.559543
6,406.762054
7,11670.879725
8,29594.203518
9,353.101631


In [198]:
np.savetxt("submit1871.csv", sub, delimiter=",")

In [ ]:
filepath = 'submit.xlsx'

df.to_excel(sub, index=False)